# Ejercicio Covid-19

In [1]:
import os
import numpy as np
import pandas as pd

#Fechas de hoy(datos España) y ayer (datos inetrnacionales)
from datetime import datetime, timedelta
from datetime import date
date_today = date.today()
date_today_complete = date.today()
date_daybefore = date_today - timedelta(days=1)
date_today = date_today.strftime('%d/%m')
date_daybefore2 = date_daybefore.strftime('%d/%m')

date_today_large = date_today_complete.strftime('%Y-%m-%d')
date_daybefore_large = date_daybefore.strftime('%Y-%m-%d')
date_today_large

'2020-05-07'

## Traer los datos de Datadista

In [2]:
data_ccaa_confirmed_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_casos.csv", sep = ',', encoding="utf-8")

data_ccaa_deaths_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_fallecidos.csv", sep = ',', encoding="utf-8")

data_ccaa_recovered_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_altas.csv", sep = ',', encoding="utf-8")

data_spain_datadista = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19.csv")


#Comprobar si están subidos los últimos datos, si no sale error es que hay datos.
data_ccaa_recovered_timeseries[date_today_large]

0      8076
1      2880
2       971
3      1345
4      1258
5      1758
6      5895
7      7149
8     23231
9       150
10     8010
11     2265
12     6959
13    38670
14      108
15     1527
16     2732
17    13070
18     2457
Name: 2020-05-07, dtype: int64

## RESUMEN: Resumen datos globales

In [3]:
# Función para obtener datos del mundo    
'''
def get_data_spain(url):    
    data_spain = pd.read_csv(url)
    data_spain = data_spain.groupby("Country/Region", as_index=False).sum()
    data_spain = data_spain.set_index("Country/Region").transpose()
    data_spain = data_spain['Spain']
    data_spain_actual = int(data_spain.iloc[-1])
    data_spain_before = int(data_spain.iloc[-2])
    data_spain_24h = data_spain_actual - data_spain_before
    return data_spain_24h, data_spain_actual
'''

def get_data_world(url):    
    data_world = pd.read_csv(url)
    data_world = data_world.groupby("Country/Region", as_index=False).sum()
    data_world = data_world.set_index("Country/Region").transpose()
    data_world_actual = int(data_world.sum(axis=1).iloc[-1])
    data_world_before = int(data_world.sum(axis=1).iloc[-2])
    data_world_24h = data_world_actual - data_world_before
    return data_world_24h, data_world_actual

data_confirmed_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
data_deaths_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
data_recovered_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"


# Para el mundo
data_confirmed_world_24h, data_confirmed_world_actual = get_data_world(data_confirmed_path)
data_deaths_world_24h, data_deaths_world_actual = get_data_world(data_deaths_path)
data_recovered_world_24h, data_recovered_world_actual = get_data_world(data_recovered_path)

''' Cogiendo datos de John Hopkins
# Para España
data_confirmed_spain_24h, data_confirmed_spain_actual = get_data_spain(data_confirmed_path)
data_deaths_spain_24h, data_deaths_spain_actual = get_data_spain(data_deaths_path)
data_recovered_spain_24h, data_recovered_spain_actual = get_data_spain(data_recovered_path)
'''

# Para España actual(dataset de datadista)
data_confirmed_spain_actual = int(data_ccaa_confirmed_timeseries[date_today_large].sum())
data_recovered_spain_actual = int(data_ccaa_recovered_timeseries[date_today_large].sum())
data_deaths_spain_actual = int(data_ccaa_deaths_timeseries[date_today_large].sum())


# Para España nuevos casos 24h(dataset de datadista)
data_spain_datadista_trans = data_spain_datadista.set_index('fecha').transpose()
data_spain_datadista_confirmed = data_spain_datadista_trans.iloc[1]
data_spain_datadista_recovered = data_spain_datadista_trans.iloc[3]
data_spain_datadista_deaths = data_spain_datadista_trans.iloc[4]
data_confirmed_spain_24h = int(data_spain_datadista_confirmed[date_today_large]) - int(data_spain_datadista_confirmed[date_daybefore_large])
data_recovered_spain_24h = int(data_spain_datadista_recovered[date_today_large]) - int(data_spain_datadista_recovered[date_daybefore_large])
data_deaths_spain_24h= int(data_spain_datadista_deaths[date_today_large]) - int(data_spain_datadista_deaths[date_daybefore_large])

# Creamos dataframes

data_spain_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_spain_actual,
                                    'Curados': data_recovered_spain_actual, 
                                    'Fallecidos': data_deaths_spain_actual}, 
                                   index=['Casos totales en España'])

data_spain_newcases = pd.DataFrame({'Confirmados': data_confirmed_spain_24h,
                                    'Curados': data_recovered_spain_24h, 
                                    'Fallecidos': data_deaths_spain_24h}, 
                                   index=['Nuevos casos en España (24h)'])


data_world_newcases = pd.DataFrame({'Confirmados': data_confirmed_world_24h,
                                    'Curados': data_recovered_world_24h, 
                                    'Fallecidos': data_deaths_world_24h}, 
                                   index=['Nuevos casos en el mundo (24h)'])

data_world_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_world_actual,
                                    'Curados': data_recovered_world_actual, 
                                    'Fallecidos': data_deaths_world_actual}, 
                                   index=['Casos totales en el mundo'])

########### Junto los datos anteriores en una tabla
data_resume_final = pd.concat([data_world_resume_actual, data_world_newcases, data_spain_resume_actual, data_spain_newcases])

data_resume_final.to_csv('data_world_resume.csv', sep = ';')
print(data_resume_final)

Confirmados  Curados  Fallecidos
Casos totales en el mundo           3662691  1198832      257239
Nuevos casos en el mundo (24h)        79636    36108        5702
Casos totales en España              256855   128511       26070
Nuevos casos en España (24h)            754     2509         213


## MUNDO: Preparación general de datos por países

In [4]:
# Datos https://github.com/CSSEGISandData/COVID-19.git
data = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )

#Saco un dataset con solo los países
data_countries = data.groupby("Country/Region", as_index=False).sum()

# Hago transposición de los datos para mover las columnas en filas 
data_trans = data_countries.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_time = data_trans[2:]

# Cogemos solo determinados países
data_time = data_time[['China', 'France', 'Germany', 'Iran','Italy', 'Korea, South', 'Norway','Spain', 'US', 'United Kingdom']]

#Hago listado de Countries
list_countries = data_time.columns
list_countries

#Convierto la columna fecha en datetime
from datetime import date
data_time.index = pd.to_datetime(data_time.index)
data_time.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-05-01,83959.0,167305.0,164077.0,95646.0,207428.0,10780.0,7783.0,213435.0,1103461.0,178685.0
2020-05-02,83959.0,168518.0,164967.0,96448.0,209328.0,10793.0,7809.0,216582.0,1132539.0,183500.0
2020-05-03,83964.0,168925.0,165664.0,97424.0,210717.0,10801.0,7847.0,217466.0,1158040.0,187842.0
2020-05-04,83966.0,169583.0,166152.0,98647.0,211938.0,10804.0,7904.0,218011.0,1180375.0,191832.0
2020-05-05,83968.0,170687.0,167007.0,99970.0,213013.0,10806.0,7955.0,219329.0,1204351.0,196243.0


## MUNDO: Confirmados, curados y fallecidos

In [5]:
######### Confirmados

data_confirmed = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )
#Saco un dataset con solo los países
data_confirmed = data_confirmed.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_confirmed = data_confirmed.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_confirmed = data_confirmed[2:]
# Cogemos solo determinados países
data_confirmed = data_confirmed[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_confirmed = data_confirmed[-1:]

######### Muertos

data_deaths = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
#Saco un dataset con solo los países
data_deaths = data_deaths.groupby("Country/Region", as_index=False).sum()
#Se ve que están todos y comprobado que suma
data_deaths[::]
# Hago transposición de los datos para mover las columnas en filas 
data_deaths = data_deaths.set_index('Country/Region').transpose()

# Y me quedo sólo con los datos temporales y número de casos
data_deaths = data_deaths[2:]
data_deaths.tail()
# Cogemos solo determinados países
data_deaths = data_deaths[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_deaths = data_deaths[-1:]

######### Recuperados

data_recovered = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv" )
#Saco un dataset con solo los países
data_recovered = data_recovered.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_recovered = data_recovered.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_recovered = data_recovered[2:]
# Cogemos solo determinados países
data_recovered = data_recovered[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_recovered = data_recovered[-1:]

######### Merge de datos

# Saco la fecha:
from datetime import datetime, timedelta
date_today_intern = date.today()
date_today_intern = date_today_intern.today() - timedelta(days=1) #Quito uno porque esta base de datos siempre muestra datos del día anterior
date_today_intern = date_today_intern.strftime('%d/%m')

#Juntar ahora datos
data_columns = ['Confirmados', 'Curados', 'Fallecidos']

data_together = pd.merge(data_confirmed, data_recovered, how='outer')
data_together = pd.merge(data_together, data_deaths, how='outer')
data_together.index = data_columns
data_together_final = data_together.transpose()
# data_together = data_together.reindex(columns =data_columns) 
# data_together = data_together.reindex(data_index)

# Exporto CSV
data_together_final.to_csv('data_world_actual.csv', sep = ';')
data_together_final

,Confirmados,Curados,Fallecidos
Country/Region,,,
China,83968.0,78870.0,4637.0
France,170687.0,52842.0,25537.0
Germany,167007.0,135100.0,6993.0
Italy,213013.0,85231.0,29315.0
Spain,219329.0,123486.0,25613.0
US,1204351.0,189791.0,71064.0
United Kingdom,196243.0,926.0,29501.0


## MUNDO: Confirmados, curados y fallecidos por habitantes de población

In [6]:
data_together

Country/Region,China,France,Germany,Italy,Spain,US,United Kingdom
Confirmados,83968.0,170687.0,167007.0,213013.0,219329.0,1204351.0,196243.0
Curados,78870.0,52842.0,135100.0,85231.0,123486.0,189791.0,926.0
Fallecidos,4637.0,25537.0,6993.0,29315.0,25613.0,71064.0,29501.0


In [7]:
pob_wuhan = 11895000
pob_china = 1393000000
pob_france = 67028048
pob_germany = 83019200
pob_italy = 60541000
pob_spain = 46940000
pob_US = 328200000
pob_UK = 66650000

# Por cada millón de habitantes hay estos casos...
data_together_den = data_together.copy()
data_together_den['China'] = (data_together_den['China'] / pob_china)*1000000 # Por cada caso infectado hay 142 que no lo están
data_together_den['France'] = (data_together_den['France'] / pob_france)*1000000 # Por cada caso infectado hay 455 que no lo están
data_together_den['Germany'] = (data_together_den['Germany'] / pob_germany)*1000000  # Por cada caso infectado hay 455 que no lo están
data_together_den['Italy'] = (data_together_den['Italy'] / pob_italy)*1000000
data_together_den['Spain'] = (data_together_den['Spain'] / pob_spain)*1000000
data_together_den['US'] = (data_together_den['US'] / pob_US)*1000000
data_together_den['United Kingdom'] = (data_together_den['United Kingdom'] / pob_UK)*1000000
data_together_den = data_together_den.round(0)
# Exporto CSV
data_together_den.to_csv('data_world_actual_den.csv', sep = ';')
data_together_den

Country/Region,China,France,Germany,Italy,Spain,US,United Kingdom
Confirmados,60.0,2547.0,2012.0,3518.0,4673.0,3670.0,2944.0
Curados,57.0,788.0,1627.0,1408.0,2631.0,578.0,14.0
Fallecidos,3.0,381.0,84.0,484.0,546.0,217.0,443.0


## MUNDO: CSV : covid19_data_drive_from100.csv

In [8]:
# Coger solo datos del último mes
data_time_30days = data_time[-200:]
# Si quiero quitar China: data_time_filter = data_time_filter.drop('China', axis=1)

#Exporto los datos en diferentes arhcivos con casa país
for country in list_countries:
    dataX = data_time_30days[country]
    dataX = dataX[dataX > 200]
    dataX.to_csv('data_' + country + ".csv", index=False, header= country)

# Lista dataframes
# dataframes_list = ['dataframe_France', 'dataframe_China', 'dataframe_Germany', 'dataframe_Iran', 'dataframe_Italy', 'dataframe_Korea', 'dataframe_Norway', 'dataframe_Spain', 'dataframe_US', 'dataframe_UnitedKingdom']

# Creamos los CSV
dataframe_France = pd.read_csv('data_France.csv')
dataframe_China = pd.read_csv('data_China.csv')
dataframe_Germany = pd.read_csv('data_Germany.csv')
dataframe_Iran = pd.read_csv('data_Iran.csv')
dataframe_Italy = pd.read_csv('data_Italy.csv')
dataframe_Korea = pd.read_csv('data_Korea, South.csv')
dataframe_Norway = pd.read_csv('data_Norway.csv')
dataframe_Spain = pd.read_csv('data_Spain.csv')
dataframe_US = pd.read_csv('data_US.csv')
dataframe_UnitedKingdom = pd.read_csv('data_United Kingdom.csv')

#Les añadimos NaN al final y reindexamos
new_index = np.arange(1,81)
dataframe_France = dataframe_France.reindex(new_index)
dataframe_China = dataframe_China.reindex(new_index)
dataframe_Germany = dataframe_Germany.reindex(new_index)
dataframe_Iran = dataframe_Iran.reindex(new_index)
dataframe_Italy = dataframe_Italy.reindex(new_index)
dataframe_Korea = dataframe_Korea.reindex(new_index)
dataframe_Norway = dataframe_Norway.reindex(new_index)
dataframe_Spain = dataframe_Spain.reindex(new_index)
dataframe_US = dataframe_US.reindex(new_index)
dataframe_UnitedKingdom = dataframe_UnitedKingdom.reindex(new_index)

#Juntar dataframes y Exportar CSV completo
data_drive_from100cases = dataframe_France.join([dataframe_China, dataframe_Germany, 
                                                 dataframe_Iran, dataframe_Italy, dataframe_Korea, 
                                                 dataframe_Norway, dataframe_Spain, dataframe_US,
                                                 dataframe_UnitedKingdom])

data_drive_from100cases.to_csv('data_world_from100.csv', sep = ';')
data_drive_from100cases.tail()

,France,China,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
76,NaN,82718.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,NaN,82809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,NaN,82883.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,82941.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NaN,83014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## MUNDO: CSV covid19_data_drive_days.csv

In [9]:
# Coger solo datos de últimos 60 días
data_time_Xdays = data_time[-30:]

#Exportar CSV por días progresivos
data_drive_lastX = data_time_Xdays.loc[:,list_countries]
data_drive_lastX.to_csv('data_world_timeline.csv', sep = ';', encoding="utf-8")
data_drive_lastX.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-05-01,83959.0,167305.0,164077.0,95646.0,207428.0,10780.0,7783.0,213435.0,1103461.0,178685.0
2020-05-02,83959.0,168518.0,164967.0,96448.0,209328.0,10793.0,7809.0,216582.0,1132539.0,183500.0
2020-05-03,83964.0,168925.0,165664.0,97424.0,210717.0,10801.0,7847.0,217466.0,1158040.0,187842.0
2020-05-04,83966.0,169583.0,166152.0,98647.0,211938.0,10804.0,7904.0,218011.0,1180375.0,191832.0
2020-05-05,83968.0,170687.0,167007.0,99970.0,213013.0,10806.0,7955.0,219329.0,1204351.0,196243.0


## ESPAÑA: Resumen datos España: data_spain_newcases24h.csv

In [10]:
data_spain_datadista

,fecha,casos_total,casos_pcr,casos_test_ac,altas,fallecimientos,ingresos_uci,hospitalizados
0,2020-02-21,2,0,NaN,NaN,NaN,0,26
1,2020-02-22,2,0,NaN,NaN,NaN,1,30
2,2020-02-23,2,0,NaN,NaN,NaN,1,33
3,2020-02-24,2,0,NaN,NaN,NaN,2,38
4,2020-02-25,3,0,NaN,NaN,NaN,2,49
...,...,...,...,...,...,...,...,...
72,2020-05-03,248028,218382,29646.0,118902.0,25264.0,10973,118495
73,2020-05-04,249120,218845,30275.0,121343.0,25428.0,10994,118889
74,2020-05-05,251081,219849,31232.0,123486.0,25613.0,11031,119609
75,2020-05-06,254050,220693,33357.0,126002.0,25857.0,11082,120466


In [11]:
# Con datos de datadista, SOLO PARA DATOS DE PAÍS
data_path_confirmed = 'casos_total'
data_path_recovered = 'altas'
data_path_deaths = 'fallecimientos'

# Evolución últimos 40 días de nuevos casos

def day_consecutive(day):
    date_cons = date.today()
    date_cons = date_cons - timedelta(days=day)
    date_cons = date_cons.strftime('%Y-%m-%d')
    return date_cons

def data_day(data_path, number_to_rest):
    data = data_spain_datadista.set_index('fecha')
    data = data[data_path]
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

data_day(data_path_confirmed, -3)

def create_dataframe(data_day_name, data_path, index_title):
    dataframe = pd.DataFrame({day_consecutive(40): data_day_name(data_path, -40),
                              day_consecutive(39): data_day_name(data_path, -39),
                              day_consecutive(38): data_day_name(data_path, -38),
                              day_consecutive(37): data_day_name(data_path, -37),
                              day_consecutive(36): data_day_name(data_path, -36),
                              day_consecutive(35): data_day_name(data_path, -35),
                              day_consecutive(34): data_day_name(data_path, -34),
                              day_consecutive(33): data_day_name(data_path, -33),
                              day_consecutive(32): data_day_name(data_path, -32),
                              day_consecutive(31): data_day_name(data_path, -31),
                              day_consecutive(30): data_day_name(data_path, -30),
                              day_consecutive(29): data_day_name(data_path, -29),
                              day_consecutive(28): data_day_name(data_path, -28),
                              day_consecutive(27): data_day_name(data_path, -27),
                              day_consecutive(26): data_day_name(data_path, -26),
                              day_consecutive(25): data_day_name(data_path, -25),
                              day_consecutive(24): data_day_name(data_path, -24),
                              day_consecutive(23): data_day_name(data_path, -23),
                              day_consecutive(22): data_day_name(data_path, -22),
                              day_consecutive(21): data_day_name(data_path, -21),
                              day_consecutive(20): data_day_name(data_path, -20),
                              day_consecutive(19): data_day_name(data_path, -19), 
                              day_consecutive(18): data_day_name(data_path, -18),
                              day_consecutive(17): data_day_name(data_path, -17),
                              day_consecutive(16): data_day_name(data_path, -16),
                              day_consecutive(15): data_day_name(data_path, -15),
                              day_consecutive(14): data_day_name(data_path, -14),
                              day_consecutive(13): data_day_name(data_path, -13),
                              day_consecutive(12): data_day_name(data_path, -12),
                              day_consecutive(11): data_day_name(data_path, -11),
                              day_consecutive(10): data_day_name(data_path, -10),
                              day_consecutive(9): data_day_name(data_path, -9),
                              day_consecutive(8): data_day_name(data_path, -8),
                              day_consecutive(7): data_day_name(data_path, -7),
                              day_consecutive(6): data_day_name(data_path, -6),
                              day_consecutive(5): data_day_name(data_path, -5),
                              day_consecutive(4): data_day_name(data_path, -4),
                              day_consecutive(3): data_day_name(data_path, -3),
                              day_consecutive(2): data_day_name(data_path, -2),
                              day_consecutive(1): data_day_name(data_path, -1),
                             }, index=index_title)
                                    
    return dataframe

data_spain_confirmed24h_datadista = create_dataframe(data_day, data_path_confirmed, ['Confirmados (24h) en España'])
data_spain_recovered24h_datadista = create_dataframe(data_day, data_path_recovered, ['Recuperados (24h) en España'])
data_spain_deaths24h_datadista = create_dataframe(data_day, data_path_deaths, ['Fallecidos (24h) en España'])
data_spain_newcases24h_datadista = pd.concat([data_spain_confirmed24h_datadista,data_spain_recovered24h_datadista,data_spain_deaths24h_datadista], axis=0)


# Exportar
data_spain_newcases24h_datadista.to_csv('data_spain_newcases24h.csv', sep = ';')
data_spain_deaths24h_datadista.to_csv('data_spain_newcases24h_deaths.csv', sep = ';')
data_spain_newcases24h_datadista

,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,...,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06
Confirmados (24h) en España,6410,5802,8146,7434,7555,7241,6680,5570,3650,5273,...,2920,3459,3128,3248,2458,1579,1092,1961,2969,2805
Recuperados (24h) en España,2323,1604,2103,3097,3486,3031,3633,3528,2022,2897,...,1673,6399,3103,2628,2570,1654,2441,2143,2516,2509
Fallecidos (24h) en España,715,696,820,929,877,845,780,670,628,757,...,301,453,268,281,276,164,164,185,244,213


In [12]:
#Con datos de John Hopkins, SOLO PARA COMPARAR CON INTERNACIONALES

path_c = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
path_d = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
path_r = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

# Evolución últimos 40 días de nuevos casos
def day_consecutive(day):
    date_cons = date.today()
    date_cons = date_cons - timedelta(days=day)
    date_cons = date_cons.strftime('%Y-%m-%d')
    return date_cons

def data_day(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Spain']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_italy(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Italy']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_france(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['France']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_germany(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Germany']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def create_dataframe(data_day_name, data_path, index_title):
    dataframe = pd.DataFrame({day_consecutive(40): data_day_name(data_path, -40),
                              day_consecutive(39): data_day_name(data_path, -39),
                              day_consecutive(38): data_day_name(data_path, -38),
                              day_consecutive(37): data_day_name(data_path, -37),
                              day_consecutive(36): data_day_name(data_path, -36),
                              day_consecutive(35): data_day_name(data_path, -35),
                              day_consecutive(34): data_day_name(data_path, -34),
                              day_consecutive(33): data_day_name(data_path, -33),
                              day_consecutive(32): data_day_name(data_path, -32),
                              day_consecutive(31): data_day_name(data_path, -31),
                              day_consecutive(30): data_day_name(data_path, -30),
                              day_consecutive(29): data_day_name(data_path, -29),
                              day_consecutive(28): data_day_name(data_path, -28),
                              day_consecutive(27): data_day_name(data_path, -27),
                              day_consecutive(26): data_day_name(data_path, -26),
                              day_consecutive(25): data_day_name(data_path, -25),
                              day_consecutive(24): data_day_name(data_path, -24),
                              day_consecutive(23): data_day_name(data_path, -23),
                              day_consecutive(22): data_day_name(data_path, -22),
                              day_consecutive(21): data_day_name(data_path, -21),
                              day_consecutive(20): data_day_name(data_path, -20),
                              day_consecutive(19): data_day_name(data_path, -19), 
                              day_consecutive(18): data_day_name(data_path, -18),
                              day_consecutive(17): data_day_name(data_path, -17),
                              day_consecutive(16): data_day_name(data_path, -16),
                              day_consecutive(15): data_day_name(data_path, -15),
                              day_consecutive(14): data_day_name(data_path, -14),
                              day_consecutive(13): data_day_name(data_path, -13),
                              day_consecutive(12): data_day_name(data_path, -12),
                              day_consecutive(11): data_day_name(data_path, -11),
                              day_consecutive(10): data_day_name(data_path, -10),
                              day_consecutive(9): data_day_name(data_path, -9),
                              day_consecutive(8): data_day_name(data_path, -8),
                              day_consecutive(7): data_day_name(data_path, -7),
                              day_consecutive(6): data_day_name(data_path, -6),
                              day_consecutive(5): data_day_name(data_path, -5),
                              day_consecutive(4): data_day_name(data_path, -4),
                              day_consecutive(3): data_day_name(data_path, -3),
                              day_consecutive(2): data_day_name(data_path, -2),
                              day_consecutive(1): data_day_name(data_path, -1),
                             }, index=index_title)
                                    
    return dataframe

'''Si se cogen los datos de España de John Hopkins
# data_spain_confirmed24h = create_dataframe(data_day, path_c, ['Confirmados (24h) en España'])
# data_spain_deaths24h = create_dataframe(data_day, path_d, ['Fallecidos (24h) en España'])
# data_spain_recovered24h = create_dataframe(data_day, path_r, ['Recuperados (24h) en España'])
#### Juntar dataframes
# data_spain_newcases24h = pd.concat([data_spain_confirmed24h,data_spain_recovered24h,data_spain_deaths24h], axis=0)
# data_spain_newcases24h.to_csv('data_spain_newcases24h.csv', sep = ';')
# data_spain_deaths24h.to_csv('data_spain_newcases24h_deaths.csv', sep = ';')
'''

######## Lo mismo en italia
data_italy_confirmed24h = create_dataframe(data_day_italy, path_c, ['Confirmados (24h) en Italia'])
data_italy_deaths24h = create_dataframe(data_day_italy, path_d, ['Fallecidos (24h) en Italia'])
data_italy_recovered24h = create_dataframe(data_day_italy, path_r, ['Recuperados (24h) en Italia'])
data_italy_newcases24h = pd.concat([data_italy_confirmed24h,data_italy_recovered24h,data_italy_deaths24h], axis=0)

######## Lo mismo en Francia
data_france_confirmed24h = create_dataframe(data_day_france, path_c, ['Confirmados (24h) en Francia'])
data_france_deaths24h = create_dataframe(data_day_france, path_d, ['Fallecidos (24h) en Francia'])
data_france_recovered24h = create_dataframe(data_day_france, path_r, ['Recuperados (24h) en Francia'])
data_france_newcases24h = pd.concat([data_france_confirmed24h,data_france_recovered24h,data_france_deaths24h], axis=0)

######## Lo mismo en Alemania
data_germany_confirmed24h = create_dataframe(data_day_germany, path_c, ['Confirmados (24h) en Alemania'])
data_germany_deaths24h = create_dataframe(data_day_germany, path_d, ['Fallecidos (24h) en Alemania'])
data_germany_recovered24h = create_dataframe(data_day_germany, path_r, ['Recuperados (24h) en Alemania'])
data_germany_newcases24h = pd.concat([data_germany_confirmed24h,data_germany_recovered24h,data_germany_deaths24h], axis=0)


###### Juntar dataframes Spain + Italy sólo de fallecidos, OJO LOS DATOS DE SPAIN SE COGEN DEL CÁLCULO ANTERIOR DE DATADISTA
data_somecountries_newcases24h_deaths = pd.concat([data_spain_deaths24h_datadista, data_italy_deaths24h,
                                                   data_france_deaths24h, data_germany_deaths24h], axis=0)
data_somecountries_newcases24h_deaths.to_csv('data_somecountries_newcases24h.csv', sep = ';')
data_somecountries_newcases24h_deaths


,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,...,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06
Fallecidos (24h) en España,715,696,820,929,877,845,780,670,628,757,...,301,453,268,281,276,164,164,185,244,213
Fallecidos (24h) en Italia,919,889,756,812,837,727,760,766,681,525,...,260,333,382,323,285,269,474,174,195,236
Fallecidos (24h) en Francia,299,320,294,419,502,882,984,1122,1054,519,...,242,437,367,427,289,218,166,137,304,333
Fallecidos (24h) en Alemania,75,91,100,112,130,145,187,168,169,140,...,99,150,188,153,156,113,76,54,127,0


## ESPAÑA: Evolución temporal casos confirmados

In [13]:
# Preparar Evolucion temporal confirmed
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries.drop(columns=['cod_ine'])
data_ccaa_confirmed_timeseries_final.rename(columns = {'CCAA': 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries_final.set_index('Comunidades Autónomas')
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries_final.iloc[0:19]

# Exportar Evolucion temporal confirmed
data_ccaa_confirmed_timeseries_final.to_csv('data_ccaa_evolution_confirmed.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')
data_ccaa_confirmed_timeseries_final

,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,...,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07
Comunidades Autónomas,,,,,,,,,,,,,,,,,,,,,
Andalucía,0,0,0,0,0,0,1,6,8,12,...,13218,13469,13656,13914,14163,14287,14349,14394,14639,14870
Aragón,0,0,0,0,0,0,0,0,0,0,...,5557,5652,5971,6103,6164,6208,6239,6282,6382,6516
Asturias,0,0,0,0,0,0,0,0,0,1,...,2688,2797,2828,2853,2864,2884,2991,3041,3073,3099
Baleares,1,1,1,1,1,1,1,1,2,2,...,1940,1950,1955,1970,1984,1990,1991,1997,2013,2023
Canarias,1,1,1,1,2,5,6,6,6,7,...,2187,2202,2205,2206,2212,2221,2225,2231,2231,2235
Cantabria,0,0,0,0,0,0,0,0,0,1,...,2386,2439,2505,2547,2577,2589,2596,2607,2626,2642
Castilla La Mancha,0,0,0,0,0,0,0,0,0,1,...,19518,19795,20071,20516,21023,21290,21511,21656,22166,22567
Castilla y León,0,0,0,0,0,0,0,0,2,2,...,19372,19851,20312,20739,20999,21229,21379,21719,22156,22537
Cataluña,0,0,0,0,0,1,2,3,5,6,...,49185,49681,49943,50334,50877,51261,51393,51798,52090,52505


## ESPAÑA: Resúmenes por comunidades autónomas

In [14]:
# SOLO HACER CUANDO LOS DATOS EN DATADISTA ESTÉN ACTUALIZADOS, SI NO, NO FUNCIONA

# Confirmados por CCAA
data_ccaa_confirmed_today = data_ccaa_confirmed_timeseries[['CCAA', date_today_large]]
data_ccaa_confirmed_today.rename(columns = {date_today_large : 'Confirmados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_today = data_ccaa_confirmed_today.set_index('Comunidades Autónomas')
data_ccaa_confirmed_today = data_ccaa_confirmed_today.copy().iloc[0:19]

# New cases
data_ccaa_confirmed_daybefore = data_ccaa_confirmed_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_confirmed_daybefore.rename(columns = {date_daybefore_large : 'Confirmados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_daybefore = data_ccaa_confirmed_daybefore.set_index('Comunidades Autónomas')
data_ccaa_confirmed_newcases = data_ccaa_confirmed_today - data_ccaa_confirmed_daybefore
data_ccaa_confirmed_newcases = data_ccaa_confirmed_newcases.copy().iloc[0:19]
data_ccaa_confirmed_newcases.rename(columns = {'Confirmados': 'Confirmados (24h)'}, inplace = True)


# Fallecidos por CCAA
data_ccaa_deaths_today = data_ccaa_deaths_timeseries[['CCAA', date_today_large]]
data_ccaa_deaths_today.rename(columns = {date_today_large : 'Fallecidos', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_deaths_today = data_ccaa_deaths_today.set_index('Comunidades Autónomas')
data_ccaa_deaths_today = data_ccaa_deaths_today.copy().iloc[0:19]

# New cases
data_ccaa_deaths_daybefore = data_ccaa_deaths_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_deaths_daybefore.rename(columns = {date_daybefore_large : 'Fallecidos', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_deaths_daybefore = data_ccaa_deaths_daybefore.set_index('Comunidades Autónomas')
data_ccaa_deaths_newcases = data_ccaa_deaths_today - data_ccaa_deaths_daybefore
data_ccaa_deaths_newcases = data_ccaa_deaths_newcases.copy().iloc[0:19]
data_ccaa_deaths_newcases.rename(columns = {'Fallecidos': 'Fallecidos (24h)'}, inplace = True)


# Recuperados por CCAA
data_ccaa_recovered_today = data_ccaa_recovered_timeseries[['CCAA', date_today_large]]
data_ccaa_recovered_today.rename(columns = {date_today_large : 'Curados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_recovered_today = data_ccaa_recovered_today.set_index('Comunidades Autónomas')
data_ccaa_recovered_today = data_ccaa_recovered_today.copy().iloc[0:19]
# New cases
data_ccaa_recovered_daybefore = data_ccaa_recovered_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_recovered_daybefore.rename(columns = {date_daybefore_large : 'Curados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_recovered_daybefore = data_ccaa_recovered_daybefore.set_index('Comunidades Autónomas')
data_ccaa_recovered_newcases = data_ccaa_recovered_today - data_ccaa_recovered_daybefore
data_ccaa_recovered_newcases = data_ccaa_recovered_newcases.copy().iloc[0:19]
data_ccaa_recovered_newcases.rename(columns = {'Curados': 'Curados (24h)'}, inplace = True)


#Juntar los tres dataframes confirmados + curados + fallecidos
data_ccaa_confirmed_recovered_deaths = pd.concat([data_ccaa_confirmed_today, data_ccaa_recovered_today, data_ccaa_deaths_today,], axis=1)

# Juntar los tres dataframes nuevos casos
data_ccaa_newcases_confirmed_recovered_deaths = pd.concat([data_ccaa_confirmed_newcases, data_ccaa_recovered_newcases, data_ccaa_deaths_newcases], axis=1)

# Exportar por grupos
data_ccaa_confirmed_recovered_deaths.to_csv('data_ccaa_confirmed_recovered_deaths.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')
data_ccaa_newcases_confirmed_recovered_deaths.to_csv('data_ccaa_newcases_confirmed_recovered_deaths.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')

data_ccaa_newcases_confirmed_recovered_deaths

,Confirmados (24h),Curados (24h),Fallecidos (24h)
Comunidades Autónomas,,,
Andalucía,231,397,13
Aragón,134,90,12
Asturias,26,18,5
Baleares,10,24,2
Canarias,4,35,1
Cantabria,16,62,1
Castilla La Mancha,401,33,30
Castilla y León,381,113,17
Cataluña,415,350,49


## ESPAÑA: Análisis por rangos de edad

In [15]:
#Comprobar si están subidos los últimos datos, si sale 0 es que no hay datos
data_spain_ccaa_age = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19_rango_edad.csv", sep = ',')
data_spain_ccaa_age[data_spain_ccaa_age["fecha"] == date_today_large].count()

fecha                33
rango_edad           33
sexo                 33
casos_confirmados    33
hospitalizados       33
ingresos_uci         33
fallecidos           33
dtype: int64

In [16]:
data_spain_ccaa_age = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19_rango_edad.csv", sep = ',', encoding = 'utf8', thousands = '.')
data_spain_ccaa_age = data_spain_ccaa_age[data_spain_ccaa_age["fecha"] == date_today_large]

def take_edad(sexo):
    data_spain_ccaa_age_all = data_spain_ccaa_age[data_spain_ccaa_age["sexo"] == sexo]
    #Para sacar datos total
    data_spain_ccaa_age_total = data_spain_ccaa_age_all.tail(1)
    data_spain_ccaa_age_total = data_spain_ccaa_age_total.iloc[:, [3,4,6]]
    data_spain_ccaa_age_total.columns = ['Confirmados','Hospitalizados', 'Fallecidos']
    #Para sacar datos concretos (de ambos, hombres y mujeres)
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_all[:-1]
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_concrete.iloc[:, [1,3,4,6]]
    data_spain_ccaa_age_concrete.columns = ['Rango de edad','Confirmados','Hospitalizados', 'Fallecidos']
    data_spain_ccaa_age_concrete.set_index('Rango de edad', inplace=True)
    return data_spain_ccaa_age_concrete, data_spain_ccaa_age_total


data_spain_ccaa_age_ambos_concrete, data_spain_ccaa_age_ambos_total = take_edad('ambos')
data_spain_ccaa_age_hombres_concrete, data_spain_ccaa_age_hombres_total = take_edad('hombres')
data_spain_ccaa_age_mujeres_concrete, data_spain_ccaa_age_mujeres_total = take_edad('mujeres')


data_spain_ccaa_age_ambos_total.index = ['Total ambos sexos']
data_spain_ccaa_age_hombres_total.index = ['Total hombres']
data_spain_ccaa_age_mujeres_total.index = ['Total mujeres']
data_spain_ccaa_age_ambos_concat = pd.concat([data_spain_ccaa_age_mujeres_total, data_spain_ccaa_age_hombres_total, 
                                             data_spain_ccaa_age_ambos_total], axis=0)

data_spain_ccaa_age_mujeres_concrete.to_csv('data_spain_ages_mujeres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_hombres_concrete.to_csv('data_spain_ages_hombres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_ambos_concat.to_csv('data_spain_ages_ambos.csv', sep = ';', encoding="utf-8")

print(data_spain_ccaa_age_ambos_concat)
print(data_spain_ccaa_age_hombres_concrete)

Confirmados  Hospitalizados  Fallecidos
Total mujeres           124164           38450        7404
Total hombres            96277           49477       10056
Total ambos sexos       220444           87929       17460
               Confirmados  Hospitalizados  Fallecidos
Rango de edad                                         
0-9                    415             141           1
10-19                  618             124           2
20-29                 4038             663          13
30-39                 7682            1865          36
40-49                13261            4876         114
50-59                17023            8044         407
60-69                16885           10168        1093
70-79                17170           12079        2938
80-89                14617            9263        4041
90 y +                4568            2254        1411


# Subir archivos a SERVIDOR desde FTP

In [17]:
# Ayuda: https://www.pythoncircle.com/post/668/uploading-a-file-to-ftp-server-using-python/
import csv
from ftplib import FTP
from io import StringIO

passw = pd.read_csv("pass.txt", sep = ',', encoding="utf-8")
host = "akakicreations.com"
username = passw.columns[0]
password = passw.columns[1]
ftp = FTP(host=host)
login_status = ftp.login(user=username, passwd=password)
print(login_status)

print(ftp.dir())

fp = open("data_world_resume.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_resume.csv"), fp, 1024)

fp = open("data_world_actual.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_actual.csv"), fp, 1024)


fp = open("data_world_actual_den.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_actual_den.csv"), fp, 1024)

fp = open("data_world_from100.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_from100.csv"), fp, 1024)

fp = open("data_world_timeline.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_timeline.csv"), fp, 1024)

fp = open("data_somecountries_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_somecountries_newcases.csv"), fp, 1024)

fp = open("data_spain_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h.csv"), fp, 1024)

fp = open("data_spain_newcases24h_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h_deaths.csv"), fp, 1024)


# Datos por CCAA
fp = open("data_ccaa_evolution_confirmed.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_confirmed.csv"), fp, 1024)

fp = open("data_ccaa_evolution_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_deaths.csv"), fp, 1024)

fp = open("data_ccaa_evolution_recovered.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_recovered.csv"), fp, 1024)

fp = open("data_ccaa_confirmed_recovered_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_confirmed_recovered_deaths.csv"), fp, 1024)

fp = open("data_ccaa_newcases_confirmed_recovered_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_newcases_confirmed_recovered_deaths.csv"), fp, 1024)

# Datos por grupos de edad
fp = open("data_spain_ages_mujeres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_mujeres.csv"), fp, 1024)

fp = open("data_spain_ages_hombres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_hombres.csv"), fp, 1024)

fp = open("data_spain_ages_ambos.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_ambos.csv"), fp, 1024)

fp.close()



230-OK. Current restricted directory is /
230 409 Kbytes used (0%) - authorized: 51200 Kb
drwxr-xr-x    2 akakicre   akakicre         4096 May  3 11:29 .
drwxr-xr-x    2 akakicre   akakicre         4096 May  3 11:29 ..
-rw-r--r--    1 akakicre   akakicre           10 May  7 16:18 .ftpquota
-rw-r--r--    1 akakicre   akakicre       154291 Apr  3 13:34 Imagen_github.jpg
-rw-r--r--    1 akakicre   akakicre          522 May  7 16:18 data_ccaa_confirmed_recovered_deaths.csv
-rw-r--r--    1 akakicre   akakicre         7045 May  7 16:18 data_ccaa_evolution_confirmed.csv
-rw-r--r--    1 akakicre   akakicre         3603 May  7 16:18 data_ccaa_evolution_deaths.csv
-rw-r--r--    1 akakicre   akakicre         3064 May  7 16:18 data_ccaa_evolution_recovered.csv
-rw-r--r--    1 akakicre   akakicre          436 May  7 16:18 data_ccaa_newcases_confirmed_recovered_deaths.csv
-rw-r--r--    1 akakicre   akakicre         1191 May  7 16:18 data_somecountries_newcases.csv
-rw-r--r--    1 akakicre   akakicre